In [1]:
import scqubits
from DiSuQ.Torch import models
from numpy import sqrt,around
from torch import tensor
from DiSuQ.Torch import dense as backend
from DiSuQ.Torch.components import complex

In [2]:
Ej = 30.02
Ec = 1.2
N = 31
ng = 1.2

### scQubits

In [3]:
transmon = scqubits.Transmon(EJ=Ej,
                              EC=Ec,
                              ng=ng,
                              ncut=31)

In [4]:
sqrt(transmon.hamiltonian()/4/Ec)

 /tmp/ipykernel_3881562/2478708122.py: 1

array([[32.2,  nan,  0. , ...,  0. ,  0. ,  0. ],
       [ nan, 31.2,  nan, ...,  0. ,  0. ,  0. ],
       [ 0. ,  nan, 30.2, ...,  0. ,  0. ,  0. ],
       ...,
       [ 0. ,  0. ,  0. , ..., 27.8,  nan,  0. ],
       [ 0. ,  0. ,  0. , ...,  nan, 28.8,  nan],
       [ 0. ,  0. ,  0. , ...,  0. ,  nan, 29.8]])

#### expansion algebra

In [5]:
basis = [31]; charge = [.25]
Q = [backend.basisQq(basis_max) for basis_max in basis]
I = [backend.identity(2*basis_max+1,complex)*charge[index]*2 for index,basis_max in enumerate(basis)]
Q_I = [backend.basisQq(basis_max)-backend.identity(2*basis_max+1,complex)*charge[index]*2 
       for index,basis_max in enumerate(basis)]
Cn_ = tensor([[1.]])

In [6]:
backend.modeMatrixProduct(Q_I,Cn_,Q_I).sqrt()/2

tensor([[30.7500+0.j,  0.0000+0.j,  0.0000+0.j,  ...,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j],
        [ 0.0000+0.j, 29.7500+0.j,  0.0000+0.j,  ...,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j, 28.7500+0.j,  ...,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j],
        ...,
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  ..., 29.2500+0.j,  0.0000+0.j,
          0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  ...,  0.0000+0.j, 30.2500+0.j,
          0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  ...,  0.0000+0.j,  0.0000+0.j,
         31.2500+0.j]])

In [7]:
H = backend.modeMatrixProduct(Q,Cn_,Q)
H -= backend.modeMatrixProduct(Q,Cn_,I)
H -= backend.modeMatrixProduct(I,Cn_,Q)
H += backend.modeMatrixProduct(I,Cn_,I)
H.sqrt()

tensor([[61.5000+0.j,  0.0000+0.j,  0.0000+0.j,  ...,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j],
        [ 0.0000+0.j, 59.5000+0.j,  0.0000+0.j,  ...,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j, 57.5000+0.j,  ...,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j],
        ...,
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  ..., 58.5000+0.j,  0.0000+0.j,
          0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  ...,  0.0000+0.j, 60.5000+0.j,
          0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  ...,  0.0000+0.j,  0.0000+0.j,
         62.5000+0.j]])

### DiSuQ

In [8]:
circuit = models.transmon([31],Ej,Ec,sparse=False)
offset = dict([(1,tensor(ng))])

In [9]:
H_LC = circuit.chargeHamiltonianLC() + circuit.chargeChargeOffset(offset)
H_J = circuit.josephsonCharge(dict())
H = H_LC + H_J
H = around(H.to_dense().detach().numpy().real,2)

In [10]:
around(sqrt(H/4/Ec),2)

 /tmp/ipykernel_3881562/1324913934.py: 1

array([[29.8,  nan,  0. , ...,  0. ,  0. ,  0. ],
       [ nan, 28.8,  nan, ...,  0. ,  0. ,  0. ],
       [ 0. ,  nan, 27.8, ...,  0. ,  0. ,  0. ],
       ...,
       [ 0. ,  0. ,  0. , ..., 30.2,  nan,  0. ],
       [ 0. ,  0. ,  0. , ...,  nan, 31.2,  nan],
       [ 0. ,  0. ,  0. , ...,  0. ,  nan, 32.2]], dtype=float32)

In [11]:
circuit.circuitEnergy(H_LC,circuit.josephsonCharge)[:5]

tensor([-21.8439,  -6.1732,   7.9766,  20.3935,  29.2915],
       grad_fn=<SliceBackward0>)

In [12]:
transmon.eigenvals()

array([-21.84386086,  -6.17324626,   7.97662279,  20.39351291,
        29.29148451,  40.98548087])